<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/BGL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#upload log files
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/X_train_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xabnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xnorm_test_index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/log2index'
!wget 'https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/reduced_index2embed'

--2023-09-04 14:32:21--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/X_train_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9341136 (8.9M) [text/plain]
Saving to: ‘X_train_index’

X_train_index       100%[===================>]   8.91M  --.-KB/s    in 0.06s   

2023-09-04 14:33:27 (137 MB/s) - ‘X_train_index’ saved [9341136/9341136]

--2023-09-04 14:33:27--  https://raw.githubusercontent.com/mostafa-ja/Anomaly-detection/main/datasets/BGL/Xabnorm_test_index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response...

In [2]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
#Json file name
with open('/content/reduced_index2embed') as f:
    embeddings = [json.loads(line) for line in f.readlines()]

len(embeddings[0])

60

In [4]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, sequence_length, dropout_prob): #input_dim= embeddings_dim
        super(LSTMAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout_prob)
        self.decoder = nn.LSTM(hidden_dim , input_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        encoded, _ = self.encoder(x)
        #print(encoded.shape)
        encoded = encoded[:,-1:,:] # output of last cell
        encoded = self.fc(encoded)
        #print(encoded.shape)
        input_decode = torch.tile(encoded, (1, self.sequence_length, 1))
        decoded, _ = self.decoder(input_decode)
        return decoded

In [5]:
sequence_length = 5
input_size = len(embeddings[0]) #embedding vector dimension
hidden_dim = 128
num_layers = 2
dropout_prob = 0.2

model = LSTMAutoencoder(input_size, hidden_dim, 2, sequence_length, dropout_prob).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(60, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (decoder): LSTM(128, 60, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=128, bias=True)
)

In [6]:
data = torch.randn(1,sequence_length, input_size) #batch_size=1
model(data.to(device)).shape

torch.Size([1, 5, 60])

In [7]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 697600


In [8]:
name = 'X_train_index'
window_size = sequence_length
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(X_train_index): 173709
Number of seqs(X_train_index): 2605635


In [12]:
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)
len(dataloader) # regarding to batch size

5090

In [13]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([512, 5, 60])


In [14]:
def adjust_learning_rate(optimizer, epoch, lr_step=(8,12,16), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [15]:
num_epochs = 20
learning_rate = 0.001

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [ ]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Autoencoder4_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [ ]:
len(embeddings)

52

In [ ]:
embeddings.append([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
embeddings[len(embeddings)-1]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
def generate(name):
    window_size = sequence_length
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            line = line + [len(embeddings)-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by 30 a template with zeros vector
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [ ]:
test_normal_loader, test_normal_length = generate('logs_ntest')
test_abnormal_loader, test_abnormal_length = generate('logs_atest')

Number of sessions(logs_ntest): 1091
Number of sessions(logs_atest): 4126


In [ ]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in test_normal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in test_abnormal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [ ]:
threshold = [0.0066,0.0068,0.0072,0.0074,0.0076]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.0066
elapsed_time: 53.111s
false positive (FP): 635, false negative (FN): 290, Precision: 96.304%, Recall: 98.278%, F1-measure: 97.281%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0068
elapsed_time: 53.198s
false positive (FP): 634, false negative (FN): 797, Precision: 96.198%, Recall: 95.267%, F1-measure: 95.730%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0072


KeyboardInterrupt: ignored

In [ ]:
threshold = [0.0057,0.0058,0.0059,0.0060,0.0061,0.0062,0.0063]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.0057
elapsed_time: 51.020s
false positive (FP): 748, false negative (FN): 77, Precision: 95.728%, Recall: 99.543%, F1-measure: 97.598%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0058
elapsed_time: 51.039s
false positive (FP): 747, false negative (FN): 77, Precision: 95.733%, Recall: 99.543%, F1-measure: 97.601%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0059
elapsed_time: 51.232s
false positive (FP): 727, false negative (FN): 83, Precision: 95.841%, Recall: 99.507%, F1-measure: 97.640%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.006
elapsed_time: 51.205s
false positive (FP): 693, false negative (FN): 135, Precision: 96.016%, Recall: 99.198%, F1-measure: 97.581%
Finished Predicting
------------------------

KeyboardInterrupt: ignored